Mount Google Drive env to import the modules and the dataset.



In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


# Append the directory to our python path using sys

Now we are able to import stuff from that directory.

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/ANFIS-imgSatellitari')
%cd /content/drive/MyDrive/ANFIS-imgSatellitari/

/content/drive/MyDrive/ANFIS-imgSatellitari


Import all the libraries and modules.



In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import torch
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from training import train_model
from testMod import test_model
import anfis
from membership import *
from utility import load_model, plot_import, split_dataset

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


cpu
Begin training.


# Test 1

In the following test I choose to configure the ANFIS training with:


*   Nr. 2 Fuzzy Sets
*   32 of Batch size
*   Nr. 100 of epochs

CLASSI NELLE METRICHE:
0 = BUILDING
1 = ROAD
2 = PAVEMENT
3 = VEGETATION
4 = BARE SOIL
5 = WATER



In [ ]:
dataset = 'datasets/reduced6ClassV3/newReduced.csv'
model = None
n_terms = 2 #IMPOSTA NUMERO DI FUZZY SET
num_categories = 6 #IMPOSTA NUMERO DI CLASSI
batch_size = 32
epoch = 100
model_l = False #SEMPRE A FALSE
hybrid = False #SEMPRE A FALSE
i = 0
lista_acc = []
k_fold = False

# Make the classes available via (controlled) reflection:
get_class_for = {n: globals()[n]
                 for n in ['BellMembFunc',
                           'GaussMembFunc',
                           'TriangularMembFunc',
                           'TrapezoidalMembFunc',
                           ]}

# DEFINIRE LA MEMBERSHIP FUNCTION ATTUALMENTE E' ABILITATA SOLO GAUSSIANA E TRIANGOLARE
membership_function = get_class_for['GaussMembFunc']

d_data, d_target = split_dataset(dataset)


# Split train into trainval-test
X_trainval, X_test, y_trainval, y_test = train_test_split(d_data, d_target, test_size=0.3, shuffle=True,
                                                          stratify=d_target, random_state=42)

# Split train into train-val
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, shuffle=True,
                                               stratify=y_trainval, random_state=42)

model = train_model(model, X_train, y_train, X_val, y_val, n_terms, num_categories,
                                   batch_size, epoch, model_l, hybrid, membership_function, i)

torch.save(model, 'models/G_model_geo_' + str(2) + '.h5')

model = torch.load('models/G_model_geo_2.h5')

test_model(model, X_test, y_test, num_categories, k_fold, i, lista_acc)

model = torch.load('models/G_model_geo_2.h5')

X_test = torch.Tensor(X_test)

y_pred = model(X_test)

cat_act = torch.argmax(y_pred, dim=1)

#PREDIZIONE SUI DATI DI TEST
print(cat_act)

cm = confusion_matrix(y_test, cat_act)
print(cm)

cl = classification_report(y_test, cat_act)
print(cl)





Model - Pre-training
AnfisNet(
  Rule  0: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  1: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf1
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  2: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  3: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf1
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  4: IF x0 is mf0 and x1 is mf1 and x2 is mf0 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  5: IF x0 is mf0 and x1 is mf1 and x2 is mf0 and x3 is mf1
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  6: IF x0 is mf0 and x1 is mf1 and x2 is mf1 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  7: IF x0 is mf0 and x1 is mf1 and x2 is mf1 and x3 is mf1
           THEN [[0.0], 

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

RISULTATI TEST
250461 of 353895 correct (accuracy=70.77%)
tensor([3, 4, 0,  ..., 3, 4, 3])
[[54484  2279  6871  4872   563   223]
 [ 6210 12778  6613  7284  3623    97]
 [10266  4492 16492  5629  2073   392]
 [ 7493  2132  1510 90933  6342  3578]
 [ 1381  1732  2577  6211 28526   205]
 [   98    93    25  8491    79 47248]]
              precision    recall  f1-score   support

         0.0       0.68      0.79      0.73     69292
         1.0       0.54      0.35      0.43     36605
         2.0       0.48      0.42      0.45     39344
         3.0       0.74      0.81      0.77    111988
         4.0       0.69      0.70      0.70     40632
         5.0       0.91      0.84      0.88     56034

    accuracy                           0.71    353895
   macro avg       0.68      0.65      0.66    353895
weighted avg       0.70      0.71      0.70    353895



# Test 2

In the following test I choose to configure the ANFIS training with:


*   Nr. 3 Fuzzy Sets
*   32 of Batch size
*   Nr. 100 of epochs



In [ ]:
dataset = 'datasets/reduced6ClassV3/newReduced.csv'
model = None
n_terms = 3 #IMPOSTA NUMERO DI FUZZY SET
batch_size = 32
num_categories = 6 #IMPOSTA NUMERO DI CLASSI
epoch = 100
model_l = False #SEMPRE A FALSE
hybrid = False #SEMPRE A FALSE
i = 0
lista_acc = []
k_fold = False

# Make the classes available via (controlled) reflection:
get_class_for = {n: globals()[n]
                 for n in ['BellMembFunc',
                           'GaussMembFunc',
                           'TriangularMembFunc',
                           'TrapezoidalMembFunc',
                           ]}

# DEFINIRE LA MEMBERSHIP FUNCTION ATTUALMENTE E' ABILITATA SOLO GAUSSIANA E TRIANGOLARE
membership_function = get_class_for['GaussMembFunc']

d_data, d_target = split_dataset(dataset)


# Split train into trainval-test
X_trainval, X_test, y_trainval, y_test = train_test_split(d_data, d_target, test_size=0.3, shuffle=True,
                                                          stratify=d_target, random_state=42)

# Split train into train-val
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, shuffle=True,
                                               stratify=y_trainval, random_state=42)


model = train_model(model, X_train, y_train, X_val, y_val, n_terms, num_categories,
                                   batch_size, epoch, model_l, hybrid, membership_function, i)

torch.save(model, 'models/G_model_geo_' + str(3) + '.h5')

model = torch.load('models/G_model_geo_3.h5')

test_model(model, X_test, y_test, num_categories, k_fold, i, lista_acc)

model = torch.load('models/G_model_geo_3.h5')

X_test = torch.Tensor(X_test)

y_pred = model(X_test)

cat_act = torch.argmax(y_pred, dim=1)

#PREDIZIONE SUI DATI DI TEST
print(cat_act)

cm = confusion_matrix(y_test, cat_act)
print(cm)

cl = classification_report(y_test, cat_act)
print(cl)





Model - Pre-training
AnfisNet(
  Rule  0: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  1: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf1
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  2: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf2
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  3: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  4: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf1
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  5: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf2
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  6: IF x0 is mf0 and x1 is mf0 and x2 is mf2 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  7: IF x0 is mf0 and x1 is mf0 and x2 is mf2 and x3 is mf1
           THEN [[0.0], 

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

RISULTATI TEST
256754 of 353895 correct (accuracy=72.55%)
tensor([3, 4, 0,  ..., 1, 4, 3])
[[58081  1916  4768  3826   604    97]
 [ 6432 14662  5733  6032  3704    42]
 [11589  3957 17031  4300  2305   162]
 [ 8697  2088  1921 89229  6963  3090]
 [ 1607  1622  2696  4748 29817   142]
 [  191   100    84  7619   106 47934]]
              precision    recall  f1-score   support

         0.0       0.67      0.84      0.75     69292
         1.0       0.60      0.40      0.48     36605
         2.0       0.53      0.43      0.48     39344
         3.0       0.77      0.80      0.78    111988
         4.0       0.69      0.73      0.71     40632
         5.0       0.93      0.86      0.89     56034

    accuracy                           0.73    353895
   macro avg       0.70      0.68      0.68    353895
weighted avg       0.72      0.73      0.72    353895



# Test 3

In the following test I choose to configure the ANFIS training with:


*   Nr. 4 Fuzzy Sets
*   32 of Batch size
*   Nr. 100 of epochs



In [ ]:
dataset = 'datasets/reduced6ClassV3/newReduced.csv'
model = None
n_terms = 4 #IMPOSTA NUMERO DI FUZZY SET
batch_size = 32
num_categories = 6 #IMPOSTA NUMERO DI CLASSI
epoch = 100
model_l = False #SEMPRE A FALSE
hybrid = False #SEMPRE A FALSE
i = 0
lista_acc = []
k_fold = False

# Make the classes available via (controlled) reflection:
get_class_for = {n: globals()[n]
                 for n in ['BellMembFunc',
                           'GaussMembFunc',
                           'TriangularMembFunc',
                           'TrapezoidalMembFunc',
                           ]}

# DEFINIRE LA MEMBERSHIP FUNCTION ATTUALMENTE E' ABILITATA SOLO GAUSSIANA E TRIANGOLARE
membership_function = get_class_for['GaussMembFunc']

d_data, d_target = split_dataset(dataset)


# Split train into trainval-test
X_trainval, X_test, y_trainval, y_test = train_test_split(d_data, d_target, test_size=0.3, shuffle=True,
                                                          stratify=d_target, random_state=42)

# Split train into train-val
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, shuffle=True,
                                               stratify=y_trainval, random_state=42)


model = train_model(model, X_train, y_train, X_val, y_val, n_terms, num_categories,
                                   batch_size, epoch, model_l, hybrid, membership_function, i)

torch.save(model, 'models/G_model_geo_' + str(4) + '.h5')

model = torch.load('models/G_model_geo_4.h5')

test_model(model, X_test, y_test, num_categories, k_fold, i, lista_acc)

model = torch.load('models/G_model_geo_4.h5')

X_test = torch.Tensor(X_test)

y_pred = model(X_test)

cat_act = torch.argmax(y_pred, dim=1)

#PREDIZIONE SUI DATI DI TEST
print(cat_act)

cm = confusion_matrix(y_test, cat_act)
print(cm)

cl = classification_report(y_test, cat_act)
print(cl)





Model - Pre-training
AnfisNet(
  Rule  0: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  1: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf1
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  2: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf2
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  3: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf3
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  4: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  5: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf1
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  6: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf2
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  7: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf3
           THEN [[0.0], 

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

RISULTATI TEST
258758 of 353895 correct (accuracy=73.12%)
tensor([3, 4, 0,  ..., 1, 4, 3])
[[57471  2235  4586  4331   601    68]
 [ 6224 15484  5197  6022  3640    38]
 [11246  4332 16776  4549  2341   100]
 [ 7861  1931  1935 91092  6660  2509]
 [ 1480  1580  2407  4971 30037   157]
 [  186    93    97  7654   106 47898]]
              precision    recall  f1-score   support

         0.0       0.68      0.83      0.75     69292
         1.0       0.60      0.42      0.50     36605
         2.0       0.54      0.43      0.48     39344
         3.0       0.77      0.81      0.79    111988
         4.0       0.69      0.74      0.72     40632
         5.0       0.94      0.85      0.90     56034

    accuracy                           0.73    353895
   macro avg       0.70      0.68      0.69    353895
weighted avg       0.73      0.73      0.72    353895

